
# Create a four chamber heart model
This example shows you how to process a case file from the Strocchi2020 database
and process that into a simulation-ready full heart model.


# Example setup
Perform the required imports
~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Import the required modules and set relevant paths, including that of the working
directory and generated model



In [ ]:
import json
import os

from ansys.heart.preprocessor.database_preprocessor import get_compatible_input
import ansys.heart.preprocessor.models as models

# specify necessary paths.
case_file = os.path.join("downloads", "Strocchi2020", "01", "01.case")


workdir = os.path.join(os.path.dirname(case_file), "FourChamber")

if not os.path.isdir(workdir):
    os.makedirs(workdir)

path_to_model = os.path.join(workdir, "heart_model.pickle")
path_to_input = os.path.join(workdir, "input_model.vtp")
path_to_part_definitions = os.path.join(workdir, "part_definitions.json")

<div class="alert alert-info"><h4>Note</h4><p>You may need to (manually) download the .case or .vtk files from the Strocchi2020
   and Rodero2021 databases first. See:

   - https://zenodo.org/records/3890034
   - https://zenodo.org/records/4590294

   Alternatively you can make use of the download
   module instead. See the download module.</p></div>



## Convert the .vtk file into compatible input



In [ ]:
input_geom, part_definitions = get_compatible_input(
    case_file, model_type="FourChamber", database="Strocchi2020"
)

# Note that the input model and part definitions can be used for later use.
# save input geometry and part definitions:
input_geom.save(path_to_input)
with open(path_to_part_definitions, "w") as f:
    json.dump(part_definitions, f, indent=True)

## Set required information
Set the right database to which this case belongs, and set other relevant
information such as the desired mesh size.



In [ ]:
info = models.ModelInfo(
    input=input_geom,
    scalar="surface-id",
    part_definitions=part_definitions,
    work_directory=workdir,
    mesh_size=1.5,
)
info.path_to_model = path_to_model


# create or clean working directory
info.create_workdir()
info.clean_workdir([".stl", ".msh.h5", ".pickle"])

## Initialize the heart model with info
Initialize the desired heart model with info.



In [ ]:
# initialize four chamber heart model
model = models.FourChamber(info)

# load input model generated in an earlier step.
model.load_input()

# mesh the volume of all structural parts.
model.mesh_volume(use_wrapper=True)

# update the model and extract the required (anatomical) features
model._update_parts()

# dump the model to disk
model.dump_model(path_to_model)

# Optionally save the simulation mesh as a vtk object for "offline" inspection
model.mesh.save(os.path.join(model.info.workdir, "simulation-mesh.vtu"))
model.save_model(os.path.join(model.info.workdir, "heart_model.vtu"))

# print some info about the processed model.
model.print_info()

# clean the working directory
info.clean_workdir(extensions_to_remove=[".stl", ".vtk", ".msh.h5"])

# dump information to stdout
info.dump_info()

# print part names
print(model.part_names)

## Visualize results
You can visualize and inspect the components of the model by accessing
various properties/attributes and invoke methods.



In [ ]:
print(f"Volume of LV cavity: {model.left_ventricle.cavity.volume} mm^3")
print(f"Volume of LV cavity: {model.left_atrium.cavity.volume} mm^3")

# plot the remeshed model
model.plot_mesh(show_edges=False)

<img src="file://_static/images/four_chamber_mesh.png" width="400pt" align="center">



In [ ]:
# plot the endocardial surface of the left ventricle.
model.left_ventricle.endocardium.plot(show_edges=True, color="r")

<img src="file://_static/images/four_chamber_lv_endocardium.png" width="400pt" align="center">



In [ ]:
# loop over all cavities and plot these in a single window.
import pyvista as pv

cavities = pv.PolyData()
for c in model.cavities:
    cavities += c.surface
cavities.plot(show_edges=True)

<img src="file://_static/images/four_chamber_cavities.png" width="400pt" align="center">

